In [1]:
!pip install accelerate>=0.20.1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
 !pip install transformers
 !pip install pytorch-lightning

In [4]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/data/news_summary_dataset_train.tsv', sep='\t')
df = train_df[:200]

In [5]:
df.isna().sum()

passage    0
summary    0
dtype: int64

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [7]:
import torch

model_name = 'hyunwoongko/kobart'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [8]:
input_tokenized = []
target_tokenized = []
for i in range(len(df)):

    input_text = df.iloc[i, 0]
    target_text = df.iloc[i, 1]

    input_ids = tokenizer(input_text, padding="max_length", max_length=1024, truncation=True)["input_ids"]
    target_ids = tokenizer(target_text, padding="max_length", max_length=1024, truncation=True)["input_ids"]

    input_tokenized.append(input_ids)
    target_tokenized.append(target_ids)

In [9]:
tokenizer.decode(input_tokenized[0])

'<s> 40억 달러 ‘딜’ 주인공 김봉진 우아한형제들 대표태풍 뒤의 고요함이랄까.\n  40억 달러짜리 ‘딜’(거래)을 마친 뒤의 사무실은 조용했다.\n  음식 배달 앱 ‘배달의민족’ 운영사 (주)우아한형제들의 서울 송파구 방이동 사옥은 비어 있었다.\n  전 직원들에게 연말 특별 휴가를 선물한 김봉진(44) 대표는 혼자 출근해 남은 일을 처리하고 있었다.\n  축하한다는 말을 건네자 “이제 시작인 걸요”라는 답이 돌아왔다.\n  그에게 기업 매각은 단순한 ‘엑시트’(창업 후 지분 매각으로 이익을 실현하는 일)가 아니었다.\n  성공 스토리 뒤에는 환희의 무게만큼 고민이 자리 잡고 있었다.\n  독일계 음식 배달서비스업체 DH(딜리버리 히어로)가 평가한 우아한형제들의 기업가치는 약 4조8000억원.\n  국내 스타트업 M&A 사상 최대 규모다.\n  앱 하나로 평가받은 기업가치가 GS나 현대건설의 시가총액과 맞먹는다.\n  ‘매각’이라는 표현을 썼지만, 창업자인 김 대표가 회사를 떠나는 것은 아니다.\n  오히려 역할이 커졌다.\n  두 회사가 절반씩 출자해 싱가포르에 세우는 합작법인 ‘우아DH아시아’의 책임자로서 아시아 11개국 사업을 총괄하게 된다.\n  DH는 우아한형제들의 투자자 지분 87%를 인수하고, 김 대표 등 경영진이 가진 지분 13%는 DH 본사 지분으로 전환하기로 했다.\n  그렇게 되면 김 대표는 DH 경영진 가운데 개인 최대 주주가 된다.\n   “더 큰 꿈 위해 글로벌 자본 선택” 김 대표의 고민은 ‘민족’이라는 단어에 닿아 있었다.\n  회사가 외국 자본에 넘어가면서 민족 브랜드가 어울리지 않게 됐다는 시선 때문이다.\n  소비자 반응이 긍정적이지만은 않다.\n “겸허하게 받아들인다.\n  DH와는 경쟁 관계이지만 창업 초기부터 지속해서 교류해왔다.\n  그 과정에서 그들이 지닌 ‘글로벌 DNA’에 놀랐다.\n  DH는 홈그라운드 격인 독일 사업마저 네덜란드 기업에 넘기고 글로벌 마케팅을 강화해왔다.\n  그들과 계속 싸울지, 합쳐서

In [10]:
tokenizer.decode(target_tokenized[0])

'<s> DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [11]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, input_tokenized, target_tokenized):
        self.input_tokenized = torch.LongTensor(input_tokenized)
        self.target_tokenized = torch.LongTensor(target_tokenized)

    def __len__(self):
        return len(self.input_tokenized)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_tokenized[idx],
            'labels': self.target_tokenized[idx]
        }


In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [13]:
from sklearn.model_selection import train_test_split

# 학습을 위해 train, test set으로 나눈다.
input_train, input_test, target_train, target_test = train_test_split(input_tokenized, target_tokenized, test_size=0.1, random_state=42)
print(len(input_train), len(input_test), len(target_train), len(target_test))

1800 200 1800 200


In [15]:
model = model.to(device)

In [14]:
from torch.utils.data import DataLoader

train_dataset = CustomDataset(input_train, target_train)
test_dataset = CustomDataset(input_test,target_test)

dl_train = DataLoader(train_dataset, batch_size=10, shuffle=False)
dl_test = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [15]:
next(iter(dl_train))['input_ids'].shape

torch.Size([10, 1024])

In [18]:
result = model(next(iter(dl_train))['input_ids'].to(device))

In [25]:
tokenizer.decode(next(iter(dl_train))['input_ids'][0], skip_special_tokens=True)

'비슷한 암 환자에게 같은 치료를 해도 어떤 환자는 효과를 보이고, 어떤 환자는 사망한다.\n  이런 차이는 환자의 식습관이나 생활 패턴에서 비롯되기도 하지만, 암 환자의 유전자가 결정적인 요인이다.\n  이에 환자의 유전자 데이터를 바탕으로 가장 적합한 항암제를 선택해 치료하는 암 환자 개인 맞춤형 의료가 대두되고 있다.\n  현재 개인 맞춤형 항암 치료는 유전체의 돌연변이에 집중되어 있다.\n  그러나 암 환자에게 일어나는 모든 돌연변이에 대해 항암제가 개발된 것이 아니기에, 이 방법을 통해 도움을 받을 수 있는 환자는 생각보다 많지 않은 편이다.\n  이를 보완하기 위해 유전체뿐만 아니라 유전자에 관련된 다른 생체 데이터를 사용하여 개인 맞춤 치료를 발전시키려는 시도가 있었으나, 환자들에게 실질적으로 도움이 될 만한 결과는 미미한 수준이었다.\n  이에 성균관대학교(총장 신동렬) 의과대학(학장 최연호) 이주상 교수 연구팀은 미국 National Cancer Institute의 Eytan Ruppin 교수 연구팀과 함께 기존의 개인 맞춤형 의료를 혁신적으로 개선할 수 있는 새로운 의료 플랫폼을 개발하였다.\n  이 플랫폼은 기존 방법과 두 가지 면에서 차별된다.\n  첫째, 암세포에서는 유전체의 돌연변이뿐만 아니라 다양한 유전적인 변이가 나타나는데, 연구진은 최근의 임상 시험 결과들을 바탕으로 유전자 발현 패턴(transcriptomics)의 변화에 초점을 맞췄다.\n  둘째, 하나의 유전자는 세포 내에서 많은 다른 유전자들과 네트워크를 이루며 긴밀한 상호작용을 한다.\n  연구진은 이러한 유전자 상호작용 중에 암 치료와 직접적으로 연결되는, 암세포의 생존에 치명적인 영향을 끼치는 유전자 상호작용(synthetic lethal interaction)을 선별하여 항암 맞춤 치료에 이용하고자 하였다.\n    현재까지 실험적인 방법으로 암 환자의 치료에 직접적인 도움을 줄 수 있는 유전자 네트워크를 밝혀내기는 쉽지 않았다.\n  따라서 연구진은 대량의 암 환자

In [23]:
tokenizer.decode(result[0][0].argmax(dim=-1), skip_special_tokens=True)

"있다. 비슷한 암 환자에게 같은 치료를 해도 어떤 환자는 효과를 보이고, 어떤 환자는 사망한다.\n  이런 차이는 환자의 식습관이나 생활 패턴에서 비롯되기도 하지만, 암 환자의 유전자가 결정적인 요인이다.\n  이에 환자의 유전자 데이터를 바탕으로 가장 적합한 항암제를 선택해 치료하는 암 환자 개인 맞춤형 의료가 대두되고 있다.\n  이에 개인 맞춤형 항암 치료가 환자의 유전 유전연변이가 집중되기도 치료  현재 개인 환자에게 일어나는 모든 돌연변이에 대해 항암제가 개발된 것이 아니기에, 이 방법을 통해 도움을 받을 수 있는 환자는 생각보다 많지 않은 편이다.\n  이를 보완하기 위해 유전체뿐만 아니라 유전자에 관련된 다른 생체 데이터를 사용하여 개인 맞춤 치료를 발전시키려는 시도가 있었으나, 환자들에게 실질적으로 도움이 될 만한 결과는 미미한 수준이었다.\n  이에 성균관대학교(총장 신동렬) 의과대학(학장 최연호) 이주상 교수 연구팀은 미국 National Cancer Institute의 Eytan Ruppin 교수 연구팀과 함께 기존의 개인 맞춤형 의료를 혁신적으로 개선할 수 있는 새로운 의료 플랫폼을 개발하였다.\n  이 플랫폼은 기존 방법과 두 가지 면에서 차별된다.\n  첫째, 암세포에서는 유전체의 돌연변이뿐만 아니라 다양한 유전적인 변이가 나타나는데, 연구진은 최근의 임상 시험 결과들을 바탕으로 유전자 발현 패턴(transcriptomics)의 변화에 초점을 맞췄다.\n  둘째, 하나의 유전자는 세포 내에서 많은 다른 유전자들과 네트워크를 이루며 긴밀한 상호작용을 한다.\n  연구진은 이러한 유전자 상호작용 중에 암 치료와 직접적으로 연결되는, 암 치료의 생존에 치명적인 영향을 끼치는 유전자 상호작용(synthetic leth  ter tion)을 선별하여 항암 맞춤 치료에 이용하고자 하였다.\n    현재까지 실험적인 방법으로 암 환자의 치료에 직접적인 도움을 줄 수 있는 유전자 네트워크를 밝혀내기는 쉽지 않았다.\n  따라서 연구진은 대량의 암 

## decoder layer는 한개로 만들고 encoder 미분값 동결

In [26]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [16]:
from transformers import BartConfig

# 주어진 설정을 불러옴
config = BartConfig.from_pretrained(model_name)

# 'decoder_layers' 값을 변경
new_decoder_layers = 1  # 새로운 값으로 변경
config.decoder_layers = new_decoder_layers

# 변경된 설정을 확인
print(config)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 1,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "kobart_version": 2.0,
  "label2id": {
    "NEGATIV

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config)
model

Some weights of the model checkpoint at hyunwoongko/kobart were not used when initializing BartForConditionalGeneration: ['decoder.layers.5.encoder_attn.v_proj.weight', 'decoder.layers.1.encoder_attn.out_proj.weight', 'decoder.layers.1.self_attn.out_proj.bias', 'decoder.layers.5.self_attn.out_proj.weight', 'decoder.layers.2.encoder_attn.v_proj.weight', 'decoder.layers.3.self_attn.q_proj.bias', 'decoder.layers.3.final_layer_norm.weight', 'decoder.layers.4.encoder_attn.k_proj.weight', 'decoder.layers.2.self_attn.out_proj.weight', 'decoder.layers.5.encoder_attn.v_proj.bias', 'decoder.layers.5.encoder_attn.out_proj.bias', 'decoder.layers.4.fc2.weight', 'decoder.layers.4.fc2.bias', 'decoder.layers.3.self_attn_layer_norm.bias', 'decoder.layers.5.self_attn.q_proj.weight', 'decoder.layers.2.fc2.bias', 'decoder.layers.4.encoder_attn.q_proj.weight', 'decoder.layers.5.self_attn.k_proj.weight', 'decoder.layers.2.final_layer_norm.bias', 'decoder.layers.5.encoder_attn.k_proj.weight', 'decoder.layers

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [18]:
result = model(next(iter(dl_train))['input_ids'].to(device))

In [19]:
tokenizer.decode(result[0][0].argmax(dim=-1), skip_special_tokens=True)

'】 유의 암 환자에게 같은 치료를 해도 어떤 환자는 효과를 보이고, 어떤 환자는 사망 한다.\n  이런 차이는 환자의 식습관이나 생활 패턴에서 비롯되기도 하지만, 암 환자의 유전자가 결정적인 요인이다.\n  이에 환자의 유전자 데이터를 바탕으로 가장 적합한 항암제를 선택해 치료하는 암 환자 개인 맞춤형 의료가 대두되고 있다.\n  현재 개인 맞춤형 항암 치료는 유전체의 돌연변이에 집중되어 있다.\n  그러나 암 환자에게 일어나는모든 돌연변이에 대해 항암제가 개발된 것이 아니기에, 이 방법을 통해 도움을 받을 수 있는 환자는 생각보다 많지 않은 편이다.\n  이를 보완하기 위해 유전체뿐만 아니라 유전자에 관련된 다른 생체 데이터를 사용하여 개인 맞춤 치료를 발전시키려는 시도가 있었으나, 환자들에게 실질적으로 도움이 될 만한 결과는 미미한 수준이었다.\n  이에 성균관대학교(총장 신동렬) 의과대학(학장 최연호) 이주상 교수 연구팀은 미국 National Cancer Institute의 Eytan Ruppin 교수 연구팀과 함께 기존의 개인 맞춤형 의료를 혁신적으로 개선할 수 있는 새로운 의료 플랫폼을 개발하였다.\n  이 플랫폼은 기존 방법과 두 가지 면에서 차별된다.\n  첫째, 암세포에서는 유전체의 돌연변이뿐만 아니라 다양한 유전적인 변이가 나타나는데, 연구진은 최근의 임상 시험 결과들을 바탕으로 유전자 발현 패턴(transcriptomics)의 변화에 초점을 맞唯  둘째, 하나의 유전자는 세포 내에서 많은 다른 유전자들과 네트워크를 이루며 긴밀한 상호작용을 한다.\n  연구진은 이러한 유전자 상호작용 중에 암 치료와 직접적으로 연결되는, 암세포의 생존에 치명적인 영향을 끼치는 유전자 상호작용(synthetic lethal interaction)을 선별하여 항암 맞춤 치료에 이용하고자 하였다.\n    현재까지 실험적인 방법으로 암 환자의 치료에 직접적인 도움을 줄 수 있는 유전자 네트워크를 밝혀내기는 쉽지 않았다.\n  따라서 연구진은 대량의 암 환자 유전

In [21]:
tokenizer.decode(next(iter(dl_train))['labels'][0], skip_special_tokens=False)

'<s> 유전체의 돌연변이에 집중되던 개인 맞춤형 항암 치료는 도움 받을 수 있는 환자가 많지 않은 편이었으며 이에 이 교수 연구팀은 새로운 의료 플랫폼을 개발했다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [19]:
len(tokenizer)

30000

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch.nn as nn

In [ ]:
tokenizer.get_vocab()['<pad>']

3

In [ ]:
tokenizer.get_vocab()['<s>']

0

In [ ]:
tokenizer.get_vocab()['</s>']

1

In [ ]:
decoder = model.get_decoder()
result = decoder(torch.tensor([[0], [1]]).to(device))

In [ ]:
next(iter(dl_train))['input_ids'].shape

torch.Size([10, 1024])

In [ ]:
input_ = nn.Embedding(30000, embedding_dim=768)(torch.zeros(9, 1).long())

In [ ]:
nn.RNN(input_size = 768, hidden_size=768, batch_first=True, dropout=0.5)(input_)[0].shape

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


torch.Size([9, 1, 768])

In [ ]:
#### 모델링
from transformers import BartConfig


class SummarizeModel(nn.Module):
    def __init__(self, model_name = 'hyunwoongko/kobart', n_hidden = 768, num_decode_layers = 1):
        super(SummarizeModel, self).__init__()
        self.config = BartConfig.from_pretrained(model_name)
        self.config.decoder_layers = num_decode_layers
        self.bart = AutoModelForSeq2SeqLM.from_pretrained(model_name, config = self.config)
        self.encoder = self.bart.get_encoder()
        self.decoder_pretrained = self.bart.get_decoder()


        # self.decode_emb = self.bart.get_decoder().embed_tokens()
        self.embedding = nn.Embedding(30000, embedding_dim=768)
        for param in self.encoder.parameters():
                param.requires_grad = False

        self.dec_cell = nn.RNN(input_size = self.encoder.config.hidden_size, hidden_size=n_hidden, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(n_hidden, 30000)

    def forward(self, input_ids, dec_input):
        encoder_outputs = self.encoder(input_ids) # bart last hidden
        encoder_last_hidden_state = encoder_outputs.last_hidden_state # (batch, input_all, hidden_size)
        encoder_last_hidden_state = encoder_last_hidden_state[:, -1, :].unsqueeze(1) # (batch, last_one, hidden_size)
        encoder_last_hidden_state = encoder_last_hidden_state.transpose(0, 1) # (last_one, batch, hidden_size))

        dec_embbedded = self.embedding(dec_input)
        # 인코더의 마지막 hidden state를 디코더의 초기 hidden state로 사용
        outputs, _ = self.dec_cell(dec_embbedded, encoder_last_hidden_state.contiguous())
        decoder_output = self.fc(outputs)

        return decoder_output


    def predict(self, input_ids):
        encoder_outputs = self.encoder(input_ids) # bart last hidden
        encoder_last_hidden_state = encoder_outputs.last_hidden_state # (batch, input_all, hidden_size)
        encoder_last_hidden_state = encoder_last_hidden_state[:, -1, :].unsqueeze(1) # (batch, last_one, hidden_size)
        encoder_last_hidden_state = encoder_last_hidden_state.transpose(0, 1) # (last_one, batch, hidden_size))

        start_token = torch.zeros(input_ids.shape[0], 1, dtype=torch.long).to(device)

        dec_input_embedded = self.embedding(start_token)

        result_tensor = torch.empty(input_ids.shape[0], 1, 30000).to(device)

        dec_hidden = encoder_last_hidden_state.contiguous()

        for i in range(1024):
            pred, dec_hidden_new = self.dec_cell(dec_input_embedded, dec_hidden)

            pred_fc = self.fc(pred)

            result_tensor = torch.cat((result_tensor, pred_fc), dim=1)

            pred_token = pred_fc.argmax(dim=-1)

            dec_input_embedded = self.embedding(pred_token)
            dec_hidden = dec_hidden_new




        result_tensor = result_tensor[:, 1:, :]

        return result_tensor




In [ ]:
model = SummarizeModel().to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
epochs = 10
loss = nn.CrossEntropyLoss()
optim = torch.optim.Adam(params=model.parameters(), lr=0.001)

for epoch in range(epochs):
    epoch_loss = 0.0
    for batch in dl_train:
        input, output = batch['input_ids'].to(device), batch['labels'].to(device)
        pred = model(input, output)
        pred = pred.transpose(0, 1)
        output = output.transpose(0, 1)
        batch_loss = 0.0
        for i in range(1024):  # input_size만큼 돌기
            batch_loss += loss(pred[i], output[i])
        batch_loss = batch_loss/1024

        epoch_loss += batch_loss

    epoch_loss = epoch_loss/len(dl_train)

    print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(epoch_loss))

    optim.zero_grad()
    epoch_loss.backward()
    optim.step()


    with torch.no_grad():
        for batch_val in dl_test:
            input_val, output_val = batch_val['input_ids'].to(device), batch_val['labels'].to(device)
            pred_val = model.predict(input_val)
            batch_loss_val = 0.0
            for i in range(1024):
                batch_loss_val += loss(pred_val[:, i, :], output_val[:, i])

            batch_loss_val = batch_loss_val/1024


            print('validation loss =', '{:.6f}'.format(batch_loss_val))


Epoch: 01 cost = 10.553007
validation loss = 4.438480
validation loss = 4.286379
Epoch: 02 cost = 3.124245
validation loss = 0.549056
validation loss = 0.535701
Epoch: 03 cost = 0.520036
validation loss = 0.429497
validation loss = 0.405455
Epoch: 04 cost = 0.367000
validation loss = 10.888442
validation loss = 10.889785
Epoch: 05 cost = 0.344299
validation loss = 11.152315
validation loss = 11.153979
Epoch: 06 cost = 0.332884
validation loss = 11.316104
validation loss = 11.317992
Epoch: 07 cost = 0.323794
validation loss = 10.756293
validation loss = 10.757447
Epoch: 08 cost = 0.315693
validation loss = 0.426108
validation loss = 0.409371
Epoch: 09 cost = 0.309340
validation loss = 0.484696
validation loss = 0.465305
Epoch: 10 cost = 0.304772
validation loss = 0.493143
validation loss = 0.472798


In [ ]:
next(iter(dl_train))['input_ids'].to(device).shape

torch.Size([10, 1024])

In [ ]:
result = model.predict(next(iter(dl_train))['input_ids'].to(device))

In [ ]:
pred_val.shape

torch.Size([10, 1024, 30000])

In [ ]:
output_val.shape

torch.Size([10, 1024])

In [ ]:
result = model(next(iter(dl_train))['input_ids'].to(device), next(iter(dl_train))['labels'].to(device))
result.shape

torch.Size([10, 1024, 30000])

In [ ]:
result = result.argmax(dim = -1)
result.shape

torch.Size([10, 1024])

In [ ]:
result[0][:10]

tensor([    0, 21087, 18081,  8981,     3,     3,     3,     3,     3,     3],
       device='cuda:0')

In [ ]:
tokenizer.decode(result[5], skip_special_tokens=True)

'증권사로부터 세계를 일─인믝Rව 직후으로로의다 원 도심 주민의갔고가글 회사~7 변'

In [ ]:
torch.save(model, "/content/drive/MyDrive/hong/kobert_news_summary_model.pt")